In [12]:
import warnings
warnings.filterwarnings( 'ignore' )
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV, TimeSeriesSplit
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.impute import SimpleImputer
import sys

In [13]:
partition = 100

In [14]:
!{sys.executable} -m pip install pytorch-tabnet --upgrade


[notice] A new release of pip is available: 24.1.2 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [15]:
trainpath = f'../../../../data/top30groups/OneHotLongLatCombined/train1/train{partition}.csv'
testpath = f'../../../../data/top30groups/OneHotLongLatCombined/test1/test{partition}.csv'

traindata = pd.read_csv(trainpath, encoding='ISO-8859-1')
testdata = pd.read_csv(testpath, encoding='ISO-8859-1')

In [16]:
import torch
print(torch.cuda.is_available())

True


In [17]:
torch.cuda.empty_cache()

In [18]:
import torch
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import LabelEncoder

def split_data(dftrain, dftest):
    Xtrain = dftrain.drop(columns=['gname']).values.astype(float)
    Ytrain = dftrain['gname'].values
    Xtest = dftest.drop(columns=['gname']).values.astype(float)
    Ytest = dftest['gname'].values

    le = LabelEncoder()
    Ytrain = le.fit_transform(Ytrain)
    Ytest = le.transform(Ytest)

    #y_pred_decoded = model.label_encoder.inverse_transform(y_pred)
    y_true_decoded = le.inverse_transform(Ytest)

    return Xtrain, Ytrain, Xtest, Ytest, y_true_decoded, le



In [19]:
from sklearn.base import BaseEstimator, ClassifierMixin
import numpy as np
from pytorch_tabnet.tab_model import TabNetClassifier

class TabNetClassifierWrapper(BaseEstimator, ClassifierMixin):
    def __init__(self, n_d=8, n_a=8, n_steps=3, gamma=1.3, lambda_sparse=1e-3, optimizer_params=None):
        self.n_d = n_d
        self.n_a = n_a
        self.n_steps = n_steps
        self.gamma = gamma
        self.lambda_sparse = lambda_sparse
        self.optimizer_params = optimizer_params or {'lr': 0.01}
        self.model = None

    def fit(self, X, y):
        self.model = TabNetClassifier(
            n_d=self.n_d,
            n_a=self.n_a,
            n_steps=self.n_steps,
            gamma=self.gamma,
            lambda_sparse=self.lambda_sparse,
            optimizer_params=self.optimizer_params,
            seed=42,
            verbose=0
        )

        self.model.fit(
            X, y,
            eval_set=[(X, y)],
            max_epochs=200,
            patience=20,
            batch_size=1024,
            virtual_batch_size=128,
            eval_metric=['accuracy']
        )

        self.classes_ = np.unique(y)  
        return self

    def predict(self, X):
        return self.model.predict(X)

    def score(self, X, y):
        preds = self.predict(X)
        return (preds == y).mean()


In [20]:
#from pytorch_tabnet.sklearn import TabNetClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedKFold
from pytorch_tabnet.tab_model import TabNetClassifier, TabNetRegressor

def find_best_tabnet(Xtrain, Ytrain, n_iter=20):
    print("Starting TabNet grid search")
    print("CUDA available:", torch.cuda.is_available())

    param_dist = {
        'n_d': [8, 16, 24],
        'n_a': [8, 16, 24],
        'n_steps': [3, 4, 5],
        'gamma': [1.0, 1.3, 1.5],
        'lambda_sparse': [1e-4, 1e-3, 1e-2],
        'optimizer_params': [{'lr': 0.01}]
    }

    random_search = RandomizedSearchCV(
        estimator=TabNetClassifierWrapper(),
        param_distributions=param_dist,
        n_iter=n_iter,
        cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42),
        scoring='accuracy',
        verbose=1,
        n_jobs=-1,
        random_state=42
    )

    random_search.fit(Xtrain, Ytrain)
    print("Best parameters:", random_search.best_params_)
    print("Best accuracy:", random_search.best_score_)

    return random_search.best_params_



In [21]:
from pytorch_tabnet.callbacks import Callback
import time

class EpochTimer(Callback):
    def __init__(self):
        self.epoch_times = []

    def on_epoch_begin(self, epoch_idx, logs=None):
        self.start_time = time.time()

    def on_epoch_end(self, epoch_idx, logs=None):
        duration = time.time() - self.start_time
        self.epoch_times.append(duration)
        print(f"⏱️ Epoch {epoch_idx + 1} took {duration:.2f} seconds")


In [22]:
Xtrain, Ytrain, Xtest, Ytest, Ytest_decoded, le = split_data(traindata, testdata)
best_tabnet_params = find_best_tabnet(Xtrain, Ytrain)

from sklearn.model_selection import train_test_split

# Split 80% train, 20% validation
X_tr, X_val, y_tr, y_val = train_test_split(
    Xtrain, Ytrain,
    test_size=0.2,
    random_state=42,
    stratify=Ytrain
)

# Re-initialize TabNet with best params
final_model = TabNetClassifier(
    **{k: v for k, v in best_tabnet_params.items()},
    verbose=1,
    seed=42
)

epoch_timer = EpochTimer()

# Retrain on full training data
final_model.fit(
    X_tr, y_tr,
    eval_set=[(X_val, y_val)],
    max_epochs=1000,
    patience=50,
    batch_size=1024,
    virtual_batch_size=128,
    callbacks=[epoch_timer]
)



Starting TabNet grid search
CUDA available: True
Fitting 5 folds for each of 20 candidates, totalling 100 fits


ValueError: 
All the 100 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
17 fits failed with the following error:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/tmp/ipykernel_23500/1674612731.py", line 27, in fit
  File "/opt/conda/lib/python3.11/site-packages/pytorch_tabnet/abstract_model.py", line 258, in fit
    self._train_epoch(train_dataloader)
  File "/opt/conda/lib/python3.11/site-packages/pytorch_tabnet/abstract_model.py", line 489, in _train_epoch
    batch_logs = self._train_batch(X, y)
                 ^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/lib/python3.11/site-packages/pytorch_tabnet/abstract_model.py", line

In [ ]:
import os
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

file_path = os.path.join("results", f"gtd{partition}.txt")

# Predict class indices for test set
y_pred = final_model.predict(Xtest)
y_proba = final_model.predict_proba(Xtest)
y_pred_decoded = le.inverse_transform(y_pred)
y_true_decoded = le.inverse_transform(Ytest)

# Make sure the directory exists
os.makedirs("results", exist_ok=True)

# Compute accuracy from decoded labels
acc = accuracy_score(y_true_decoded, y_pred_decoded)

# Write metrics to file
with open(file_path, "w") as file:
    file.write(f"Accuracy: {acc:.4f}\n")
    file.write(f"Precision weighted: {precision_score(y_true_decoded, y_pred_decoded, average='weighted'):.4f}\n")
    file.write(f"Recall weighted: {recall_score(y_true_decoded, y_pred_decoded, average='weighted'):.4f}\n")
    file.write(f"F1 Score weighted: {f1_score(y_true_decoded, y_pred_decoded, average='weighted'):.4f}\n")
    file.write(f"Precision micro: {precision_score(y_true_decoded, y_pred_decoded, average='micro'):.4f}\n")
    file.write(f"Recall micro: {recall_score(y_true_decoded, y_pred_decoded, average='micro'):.4f}\n")
    file.write(f"F1 Score micro: {f1_score(y_true_decoded, y_pred_decoded, average='micro'):.4f}\n")
    file.write(f"Precision macro: {precision_score(y_true_decoded, y_pred_decoded, average='macro'):.4f}\n")
    file.write(f"Recall macro: {recall_score(y_true_decoded, y_pred_decoded, average='macro'):.4f}\n")
    file.write(f"F1 Score macro: {f1_score(y_true_decoded, y_pred_decoded, average='macro'):.4f}\n")
    file.write(f"roc auc weighted: {roc_auc_score(y_true_decoded, y_proba, multi_class='ovr', average='weighted'):.4f}\n")
    file.write(f"roc auc macro: {roc_auc_score(y_true_decoded, y_proba, multi_class='ovr', average='macro'):.4f}\n")
    file.write(f"roc auc micro: {roc_auc_score(y_true_decoded, y_proba, multi_class='ovr', average='micro'):.4f}\n")

with open(f"results/epoch_time_gtd{partition}.txt", "w") as f:
    f.write('\n'.join(str(x) for x in epoch_timer.epoch_times))



In [ ]:
print(classification_report(Ytest_decoded, y_pred_decoded))

                                                  precision    recall  f1-score   support

                          Abu Sayyaf Group (ASG)       0.89      0.57      0.69        30
        African National Congress (South Africa)       1.00      1.00      1.00        30
                                Al-Qaida in Iraq       0.57      0.53      0.55        30
        Al-Qaida in the Arabian Peninsula (AQAP)       0.83      0.63      0.72        30
                                      Al-Shabaab       1.00      0.90      0.95        30
             Basque Fatherland and Freedom (ETA)       0.75      0.20      0.32        30
                                      Boko Haram       0.58      0.93      0.72        30
  Communist Party of India - Maoist (CPI-Maoist)       0.72      0.97      0.83        30
       Corsican National Liberation Front (FLNC)       0.51      0.87      0.64        30
                       Donetsk People's Republic       0.91      1.00      0.95        30
Farabundo

In [ ]:
def plot_confusion_matrix(y_true, y_pred, labels):
    from sklearn.metrics import confusion_matrix
    import matplotlib.pyplot as plt
    import seaborn as sns
    import numpy as np

    cm = confusion_matrix(y_true, y_pred, labels=labels)
    cm_normalized = cm.astype('float') / cm.sum(axis=1, keepdims=True)

    plt.figure(figsize=(18, 16))
    sns.heatmap(cm_normalized,
                annot=True,
                fmt=".2f",
                xticklabels=labels,
                yticklabels=labels,
                cmap="viridis",
                square=True,
                linewidths=0.5,
                cbar_kws={"shrink": 0.8})

    plt.title(f"Normalized Confusion Matrix", fontsize=18)
    plt.xlabel("Predicted Label", fontsize=14)
    plt.ylabel("True Label", fontsize=14)
    plt.xticks(rotation=90)
    plt.yticks(rotation=0)
    plt.tight_layout()

    # Save the figure
    save_path = f"results/confusion_matrix_partition_{partition}.png"
    plt.savefig(save_path, dpi=300)
    plt.close()

    print(f"Saved confusion matrix for partition {partition} to {save_path}")


In [ ]:
# Get all unique class labels from the truths
class_labels = np.unique(Ytest_decoded)

plot_confusion_matrix(Ytest_decoded, y_pred_decoded, labels=class_labels)



Saved confusion matrix for partition 100 to results/confusion_matrix_partition_100.png
